# Kernel Semantic cu Integrarea Serverului OpenBnB MCP

Acest notebook demonstrează cum să utilizezi Kernel Semantic împreună cu serverul OpenBnB MCP pentru a căuta cazare reală pe Airbnb folosind MCPStdioPlugin. Pentru accesul la LLM, se folosește Azure AI Foundry. Pentru a configura variabilele de mediu, poți urma [Lecția de Configurare](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Crearea conexiunii pluginului MCP

Ne vom conecta la [serverul MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) folosind MCPStdioPlugin. Acest server oferă funcționalitatea de căutare Airbnb prin intermediul pachetului @openbnb/mcp-server-airbnb.


## Crearea Clientului

În acest exemplu, vom folosi Azure AI Foundry pentru accesul nostru la LLM. Asigură-te că variabilele de mediu sunt configurate corect.


## Configurarea Mediului

Configurează setările Azure OpenAI. Asigură-te că ai setat următoarele variabile de mediu:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Înțelegerea integrării OpenBnB MCP

Acest notebook se conectează la **serverul real OpenBnB MCP**, care oferă funcționalități reale de căutare Airbnb.

### Cum funcționează:

1. **MCPStdioPlugin**: Utilizează comunicarea prin intrare/ieșire standard cu serverul MCP
2. **Pachet NPM real**: Descarcă și rulează `@openbnb/mcp-server-airbnb` prin npx
3. **Date în timp real**: Returnează date reale despre proprietăți Airbnb din API-urile lor
4. **Descoperirea funcțiilor**: Agentul descoperă automat funcțiile disponibile de la serverul MCP

### Funcții disponibile:

Serverul OpenBnB MCP oferă, de obicei:
- **search_listings** - Caută proprietăți Airbnb după locație și criterii
- **get_listing_details** - Obține informații detaliate despre proprietăți specifice
- **check_availability** - Verifică disponibilitatea pentru date specifice
- **get_reviews** - Recuperează recenzii pentru proprietăți
- **get_host_info** - Obține informații despre gazdele proprietăților

### Cerințe preliminare:

- **Node.js** instalat pe sistemul tău
- **Conexiune la internet** pentru a descărca pachetul serverului MCP
- **NPX** disponibil (vine împreună cu Node.js)

### Testarea conexiunii:

Poți testa manual serverul MCP rulând:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Aceasta va descărca și porni serverul OpenBnB MCP, la care Semantic Kernel se conectează ulterior pentru a accesa date reale Airbnb.


## Rularea Agentului cu Serverul OpenBnB MCP

Acum vom rula Agentul AI care se conectează la serverul OpenBnB MCP pentru a căuta cazare reală pe Airbnb în Stockholm pentru 2 adulți și 1 copil. Simțiți-vă liber să modificați lista `user_inputs` pentru a schimba criteriile de căutare.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Rezumat  
Felicitări! Ai reușit să construiești un agent AI care se integrează cu căutarea de cazare din lumea reală folosind Protocolul de Context al Modelului (MCP):  

Tehnologii utilizate:  
- Semantic Kernel - Pentru construirea agenților inteligenți cu Azure OpenAI  
- Azure AI Foundry - Pentru capabilități LLM și completarea conversațiilor  
- MCP (Model Context Protocol) - Pentru integrarea standardizată a instrumentelor  
- OpenBnB MCP Server - Pentru funcționalitatea reală de căutare Airbnb  
- Node.js/NPX - Pentru rularea serverului MCP extern  

Ce ai învățat:  
- Integrarea MCP: Conectarea agenților Semantic Kernel la servere MCP externe  
- Acces la date în timp real: Căutarea proprietăților Airbnb reale prin API-uri live  
- Comunicare prin protocol: Utilizarea comunicării stdio între agent și serverul MCP  
- Descoperirea funcțiilor: Descoperirea automată a funcțiilor disponibile de la serverele MCP  
- Răspunsuri în flux: Capturarea și logarea apelurilor de funcții în timp real  
- Redare HTML: Formatarea răspunsurilor agentului cu tabele stilizate și afișaje interactive  

Pași următori:  
- Integrarea altor servere MCP (vreme, zboruri, restaurante)  
- Construirea unui sistem multi-agent care combină protocoalele MCP și A2A  
- Crearea de servere MCP personalizate pentru propriile surse de date  
- Implementarea memoriei conversaționale persistente între sesiuni  
- Implementarea agentului în Azure Functions cu orchestrarea serverului MCP  
- Adăugarea autentificării utilizatorilor și a capabilităților de rezervare  

Avantaje cheie ale arhitecturii MCP:  
- Standardizare: Protocol universal pentru conectarea agenților AI la instrumente externe  
- Date în timp real: Acces la informații live, actualizate, din diverse servicii  
- Extensibilitate: Integrare ușoară a noilor surse de date și instrumente  
- Interoperabilitate: Funcționează pe diferite cadre AI și platforme de agenți  
- Separarea preocupărilor: Distincție clară între logica AI și accesul la date externe  



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
